In [1]:
from skimage import io, color, transform
import numpy as np
import time
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
# Read data set
data_dir = '/Users/kunpenghao/Desktop/Study_in_SIT/EE_628/project'
test_str = data_dir + '/data/test1/*.jpg'
train_str = data_dir + '/data/train/*.jpg'

def convert_gray(f): 
    rgb = io.imread(f)
    rgb = transform.resize(rgb, (28, 28))
    return color.rgb2gray(rgb)

test_im = io.ImageCollection(test_str, load_func = convert_gray)
train_im = io.ImageCollection(train_str, load_func = convert_gray)

train = [0 for i in range(25000)]
test = [0 for i in range(12500)]

time_start = time.time()
for i in range(len(train_im)):
    pic = train_im[i]
    train[i] = pic
for i in range(len(test_im)):
    pic = test_im[i]
    test[i] = pic
time_end = time.time()
print(time_end - time_start)

train = np.array(train)
test = np.array(test)

print(train.shape)
print(test.shape)

/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


185.97323393821716
(25000, 28, 28)
(12500, 28, 28)


In [3]:
# Make class for train_set
# x_train(20000), 0-9999 cat; 10000-19999 dog
# x_val(5000), 0-2499 cat; 2500-4999 dog
# y_train(20000) y_val(5000), cat = 1, dog = 0
cat_train = train[:10000]
cat_val = train[10000:12500]
dog_train = train[12500:22500]
dog_val = train[22500:]
x_train = np.zeros([20000, 28,28])
x_val = np.zeros([5000, 28,28])
x_train[:10000] = cat_train
x_train[10000:] = dog_train
x_val[:2500] = cat_val
x_val[2500:] = dog_val
y_train = np.zeros([20000, 1])
y_val = np.zeros([5000, 1])
y_train[10000:] = 1
y_val[2500:] = 1

In [4]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_val = x_val.reshape(x_val.shape[0], 28, 28, 1)
x_test = test.reshape(test.shape[0], 28, 28, 1)

In [5]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape
from keras.models import Model, Sequential
from keras import regularizers

autoencoder = Sequential()

# Encoder Layers
autoencoder.add(Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=x_train.shape[1:]))
autoencoder.add(MaxPooling2D((2, 2), padding='same'))
autoencoder.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
autoencoder.add(MaxPooling2D((2, 2), padding='same'))
autoencoder.add(Conv2D(8, (3, 3), strides=(2,2), activation='relu', padding='same'))

autoencoder.add(Flatten())
autoencoder.add(Dense(2, activation='sigmoid'))
autoencoder.add(Dense(128, activation='relu'))
autoencoder.add(Reshape((4, 4, 8)))

# Decoder Layers
autoencoder.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(16, (3, 3), activation='relu'))
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(1, (3, 3), activation='sigmoid', padding='same'))

autoencoder.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 8)           584       
_________________________________________________________________
flatten_1 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
__________

In [6]:
encoder = Sequential()
encoder.add(autoencoder.layers[0])
encoder.add(autoencoder.layers[1])
encoder.add(autoencoder.layers[2])
encoder.add(autoencoder.layers[3])
encoder.add(autoencoder.layers[4])
encoder.add(autoencoder.layers[5])
encoder.add(autoencoder.layers[6])
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 8)           584       
_________________________________________________________________
flatten_1 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
Total para

In [7]:
from keras.optimizers import Adam
autoencoder.compile(loss='mse', optimizer=Adam())
autoencoder.fit(x_train, x_train,
                epochs=20,
                batch_size=512,
                shuffle=True,
                validation_data=(x_val, x_val))

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 37s 2ms/step - loss: 0.0635 - val_loss: 0.0633
Epoch 2/20
20000/20000 [==============================] - 35s 2ms/step - loss: 0.0577 - val_loss: 0.0550
Epoch 3/20
20000/20000 [==============================] - 34s 2ms/step - loss: 0.0527 - val_loss: 0.0523
Epoch 4/20
20000/20000 [==============================] - 34s 2ms/step - loss: 0.0509 - val_loss: 0.0510
Epoch 5/20
20000/20000 [==============================] - 34s 2ms/step - loss: 0.0496 - val_loss: 0.0489
Epoch 6/20
20000/20000 [==============================] - 35s 2ms/step - loss: 0.0465 - val_loss: 0.0457
Epoch 7/20
20000/20000 [==============================] - 34s 2ms/step - loss: 0.0451 - val_loss: 0.0451
Epoch 8/20
20000/20000 [==============================] - 34s 2ms/step - loss: 0.0447 - val_loss: 0.0447
Epoch 9/20
20000/20000 [==============================] - 34s 2ms/step - loss: 0.0444 - val_loss: 0.0446
Epoch 

In [9]:
import pandas as pd
predictions = encoder.predict_classes(x_test)
predictions = list(predictions)
pic = []
for i in range(12500):
    pic.append(i+1)

sub = pd.DataFrame({'id': pic,'label': predictions})
sub.to_csv('./CNN_AE_output.csv', index=False)